<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Восстановление-исходных-данных" data-toc-modified-id="Восстановление-исходных-данных-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Восстановление исходных данных</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Вывод</a></span>

# Защита персональных данных клиентов

Необходимо защитить данные клиентов страховой компании таким образом, чтобы персональную информацию о клиентах было сложно восстановить, но вместе с тем зашифрованные данные не влияли на качество модели машинного обучения.

## Загрузка данных

In [ ]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
# прочитаем таблицу и сохраним данные в переменную data
data = pd.read_csv('/datasets/insurance.csv')

In [ ]:
# выведем первые пять строк таблицы
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


По условиям задания:

**Признаки:** пол, возраст и зарплата застрахованного, количество членов его семьи.

**Целевой признак:** количество страховых выплат клиенту за последние 5 лет.

In [ ]:
# выведем общую информацию по таблице (кол-во ненулевых значений, типы данных)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Данные в столбцах `Возраст` и `Зарплата` представлены в виде float, остальные - в int, необходимо привести к единообразию. Также можем заметить, что в данных нет пропусков.

In [ ]:
# приведем все данные таблицы к типу int
data[['Возраст', 'Зарплата']] = data[['Возраст', 'Зарплата']].astype('int64')

In [ ]:
# проверим данные на наличие дубликатов
data.duplicated().sum()

153

In [ ]:
data[data.duplicated()]

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
281,1,39,48100,1,0
488,1,24,32900,1,0
513,0,31,37400,2,0
718,1,22,32600,1,0
785,0,20,35800,0,0
...,...,...,...,...,...
4793,1,24,37800,0,0
4902,1,35,38700,1,0
4935,1,19,32700,0,0
4945,1,21,45800,0,0


С одной стороны - не факт, что это дубликаты, ведь в теории могут быть у страховой компании клиенты одного возраста с одинаковым уровнем дохода. С другой стороны, даже если это не дубликаты, на качество обучения модели эти данные не повлияют.

upd: проверила опытным путем, точно не повлияет.

In [ ]:
# посмотрим на распределение значений в столбцах. Какие-то дополнительные графики на поиск выбросов в данном случае нет 
# смысла строить
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.359400,1.194200,0.148000
std,0.500049,8.440807,9900.082063,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Выбросов в данных нет.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** При умножении матрицы признаков на обратимую матрицу качество линейной регрессии не изменится.

**Обоснование:** Докажем наш ответ на основе формулы ~~вычисления предсказаний для каждой из матриц.~~ расчета весов для линейной регрессии.

Представим новую матрицу признаков $X'$ как произведение исходной $X$ на обратимую матрицу $P$:

$$
X' = X*P
$$

Следовательно, формула расчета весов для линейной регрессии будет выглядеть следующем образом:

$$
w' = (X'^{T}*X')^{-1}*X'^{T}*y = ((X*P)^{T}*X*P)^{-1}*(X*P)^{T}*y
$$

По свойствам матриц $(X*P)^{T} = P^{T}*X^{T}$, следовательно:

$$
w' = (P^{T}*X^{T}*X*P)^{-1}*P^{T}*X^{T}*y
$$

По свойству ассоциативноси матриц $(A*(B*C) = ((A*B)*C)$, следовательно, можем представить формулу следующим образом:

$$
w' = (P^{T}*(X^{T}*X)*P)^{-1}*P^{T}*X^{T}*y
$$

Раскроем скобку $(P^{T}*(X^{T}*X)*P)^{-1}$:

$$
w' = P^{-1}*(X^{T}*X)^{-1}*(P^{T})^{-1}*P^{T}*X^{T}*y
$$

Так как $P$ по условию обратимая, то произведение $(P^{T})^{-1}*P^{T}$ равно единичной матрице - $E$:

$$
w' = P^{-1}*(X^{T}*X)^{-1}*E*X^{T}*y = P^{-1}*(X^{T}*X)^{-1}*X^{T}*y
$$

Как можем заметить, справа получилась формула для расчета $w$:

$$
w' = P^{-1}*w
$$

Теперь обратимся к формуле рассчета предсказаний для линейной регрессии - $a$:

$$
a = X*w
$$
$$
a' = X'*w' = X*P*P^{-1}*w
$$

Как мы уже ранее заметили, так как $P$ по условию обратимая, то произведение $(P^{T})^{-1}*P^{T}$ равно единичной матрице - $E$:

$$
a' = X*E*w
$$

Но так как любая матрица $A$ при умножении на единичную матрицу равна этой же самой матрице $A$:

$$
a' = X*w = a
$$

Мы доказали, что предсказания $a'$ для матрицы признаков, умноженных на обратимую матрицу равны предсказаниям $a$. Следовательно, при умножении признаков на обратимую матрицу - качество линейной регрессии не изменится.

## Алгоритм преобразования

**Алгоритм**

Для защиты данных клиентов нам необходмо создать обратимую матрицу, которая бы шифровала данные. Однако также важно иметь возможность потом зашифрованные данные расшифровать, если возникнет такая необходимость. В связи с этим предлагаем создать класс, который:
- принимает на вход исходную матрицу признаков, а возвращает уже зашифрованную (умноженную на случайную обратимую матрицу);
- принимает на вход зашифрованную матрицу, а возвращает исходную (декодированную).

Следовательно алгоритм действий для данного раздела следующий:

- создать класс для шифровки/дешифровки данных о клиентах;
- обучить модель на исходных данных и найти метрику R2;
- с помощью созданного объекта зашифровать данные, обучить на них модель и также найти метрику R2;
- сравнить полученные метрики и сделать выводы.

**Обоснование**

На прошлом этапе исследования мы выяснили, что умножение исходной матрицы признаков на обратимую матрицу никак не влияет на качество предсказаний модели, следовательно, умножение матрицы на случайным образом сгенерированную обратимую матрицу решит две наши задачи - зашифрует данные и не повлияет на работу модели.

## Проверка алгоритма

In [ ]:
# объявим класс для шифровки/расшифровки данных о клиентах
class DataEncoder:
    # фиксация рандома при инициализации
    def __init__(self, random_state=None):
        self.__rstate = random_state
        
    def fit(self, features):
        self.__inverse(features)
        return self
    
    def __inverse(self, features):
        np.random.seed(self.__rstate)
        # генерирует случайную матрицу, при исп. функции numpy.random.normal() вероятность получить необратимую матрицу 
        # близка к нулю
        self.__matrix = np.random.normal(size=(features.shape[1], features.shape[1]))
        # генерирует обратную матрицу. Также эта фун-ия проверяет сгенерированную матрицу на обратимость, если матрица
        # необратима, то будет ошибка
        self.__matrix_inv = np.linalg.inv(self.__matrix)
        # преобразует данные, умножив их на случайную квадратную обратимую матрицу    
    def transform(self, features):
        return pd.DataFrame(np.dot(features, self.__matrix))
    
    def fit_transform(self, features):
        self.fit(features)
        return self.transform(features)
    # декодирует данные через умножение матрицы преобразованных признаков на обратную случайную обратимую матрицу
    def decode(self, transformed_features):
        reversed = np.dot(transformed_features, self.__matrix_inv)
    # результат дешифровки данных выводим в виде датафрейма, данные округляем и приводим к типу int    
        return pd.DataFrame(np.dot(transformed_features, self.__matrix_inv)).round(2).astype('int64')

In [ ]:
# разделим данные на признаки (features) и целевой признак (target)
features = data.drop(columns=['Страховые выплаты'])
target = data['Страховые выплаты']

In [ ]:
# обучим наш преобразователь данных на признаках и трансформируем их
Encoder = DataEncoder()
Encoder.fit(features)
transformed_features = Encoder.transform(features)

In [ ]:
# разделим признаки и цель на обучающую и тестовую выборки
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.4,
                                                                                            random_state=12345)

# то же самое сделаем для преобразованных признаков
transformed_features_train, transformed_features_test, transformed_target_train, transformed_target_test = train_test_split(transformed_features, target, test_size=0.4,
                                                                                            random_state=12345)

In [ ]:
#создадим модель, которую обучим на исходных данных 
model_for_original_data = LinearRegression()
# а эту обучим уже на преобразованных данных
model_for_transformed_data = LinearRegression()

model_for_original_data.fit(features_train, target_train)
model_for_transformed_data.fit(transformed_features_train, transformed_target_train)

# найдем предсказания для тестовой выборки исходных данных
prediction1 = model_for_original_data.predict(features_test)
# найдем предсказания для тестовой выборки преобразованных данных
prediction2 = model_for_transformed_data.predict(transformed_features_test)

In [ ]:
# найдем коэф. детерминации для исх. данных
model_for_original_data_r2 = r2_score(target_test, prediction1)
# найдем коэф. детерминации для преобразованных данных
model_for_transformed_data_r2 = r2_score(transformed_target_test, prediction2)
print(f'R2 score на исходных данных: {model_for_original_data_r2:.5f}')
print(f'R2 score на преобразованных данных: {model_for_transformed_data_r2:.5f}')

R2 score на исходных данных: 0.42375
R2 score на преобразованных данных: 0.42375


In [ ]:
# проверим равны ли метрики
model_for_original_data_r2.round(5) == model_for_transformed_data_r2.round(5)

True

## Восстановление исходных данных

In [ ]:
# дешифруем данные
recovered_features = Encoder.decode(transformed_features)
recovered_features.columns = features.columns
recovered_features

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0
...,...,...,...,...
4995,0,28,35700,2
4996,0,34,52400,1
4997,0,20,33900,2
4998,1,22,32700,3


In [ ]:
# сравним восстановленные данные и исходные, равны ли они между собой
recovered_features == features

,Пол,Возраст,Зарплата,Члены семьи
0,True,True,True,True
1,True,True,True,True
2,True,True,True,True
3,True,True,True,True
4,True,True,True,True
...,...,...,...,...
4995,True,True,True,True
4996,True,True,True,True
4997,True,True,True,True
4998,True,True,True,True


In [ ]:
# восстановим датафрейм полностью
recovered_features[4] = target
recovered_data = recovered_features
recovered_data.columns = data.columns
recovered_data

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0
...,...,...,...,...,...
4995,0,28,35700,2,0
4996,0,34,52400,1,0
4997,0,20,33900,2,0
4998,1,22,32700,3,0


## Вывод

В ходе работы мы на практическом примере доказали, что при умножении матрицы с признаками размером *m x n* на случайную обратимую матрицу *n x n* (т.е. квадратную), качество линейной регрессии не изменится. 

Для доказательства данной гипотезы мы объявили объект, который шифрует исходные данные и дешифрует преобразованные за счет умножения матрицы признаков в первом случае на сгенерированную случайную обратимую матрицу, а во втором - на обратную данной сгенерированной матрице матрицу. 

Такое шифрование можно использовать для защиты персональных данных, т.к. по преобразованному датафрейму будет тяжело восстановить характер и значения исходных данных.

Если рассматривать такого рода шифрование для предотвращения передачи данных третьим лицам, то к преимуществам данного подхода можно отнести, в первую очередь, ощутимую экономию ресурсов по сравнению с некоторыми известными алгоритмами шифрования, в особенности это касается ассиметричных алгоритмов с достаточно длинными ключами (AES, RSA), которые требуют много процессорного времени.


К главному недостатку подхода можно отнести увеличение сложности проверки на вырожденность преобразования с ростом количества признаков в выборке. В рассмотренном случае необходимо подобрать подходящую матрицу размера 4×4 . Однако, если признаки будут исчисляться тысячами, то расчет соответствующих определителей может быть крайне ресурсозатратным. 